In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,1998-02-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,4.0,0,10.0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,NaN,0,0.0,NaN,0.0,None,612.0,NaN,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,None,"OCEANSIDE, CITY OF",97.0,1.0,1.0,937.0,NaN,A,0,0.0,None,NaN,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,4053d562-94c7-4155-9d60-3030f34a7abc
1,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,NaN,0,0.0,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,6.0,6.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,4094445e-bc54-42b2-a60d-b949bdc2df3a
2,0,2020-01-22T16:55:53.000Z,NaN,1,9.0,1998-09-28T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,2220.0,0,1720.0,0.0,0.0,None,133.0,NaN,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,100000.0,0.0,A,0,0.0,None,NaN,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,659ba8e0-1bb3-4442-ad57-3f384db43176
3,0,2019-09-19T13:45:58.000Z,1.0,1,9.0,1994-10-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,NaN,0,0.0,NaN,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,450026.0,None,"BEAUFORT, CITY OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,97063578-59b6-403d-b9c3-2e70fd8b6f38
4,0,2019-09-19T13:45:58.000Z,NaN,1,8.0,1996-03-11T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,3115.0,0,1545.0,0.0,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,125092.0,None,BREVARD COUNTY *,NaN,NaN,1.0,0.0,0.0,A,0,0.0,None,1.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,613c20ef-7838-48ee-a257-07a49e997d3a


In [4]:
#fill na values with 0

df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].fillna(0)
df_geographic_unique['countyCode'] = df_geographic_unique['countyCode'].fillna(0)
df_geographic_unique['censusTract'] = df_geographic_unique['censusTract'].fillna(0)
df_geographic_unique['censusBlockGroupFips'] = df_geographic_unique['censusBlockGroupFips'].fillna(0)
df_geographic_unique['latitude'] = df_geographic_unique['latitude'].fillna(0)
df_geographic_unique['longitude'] = df_geographic_unique['longitude'].fillna(0)

df_geographic_unique = df_geographic_unique.drop_duplicates()

In [5]:
#convert the units to string to use mapping and fix the lenght of each units

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].astype(int).astype(str)
df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censusBG.zfill(5) for censusBG in df_geographic_unique['countyCode']]

df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [6]:
# Define bins and labels for yearOfLoss_1990_2021

bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique.drop(columns='yearOfLoss', inplace=True)

df_geographic_unique = df_geographic_unique.drop_duplicates()

## Read shapefiles

In [7]:
#State shapefile

states = pygris.states()

state_df = states[['STUSPS', 'NAME', 'geometry']]

Using the default year of 2021


In [8]:
#Lat-Long shapefile
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [9]:
# Blockgroup shapefile

chunk_size = 40000 
chunks = [x for x in range(0, 120000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df= pd.concat(gdf_list, ignore_index=True)

In [10]:
# Read shapefile of zipcode

chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 80000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

In [11]:
# Read shapefile of county code
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [12]:
# Read shapefile of census tract code
chunk_size = 30000 
chunks = [x for x in range(0, 60000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/Tract_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
Tract_df= pd.concat(gdf_list, ignore_index=True)

### Mapping Geometries to the Dataset

In [13]:
state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)
Tract_df.rename(columns={'geometry': 'geometry_tract'}, inplace=True)

In [14]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

lat_long_df.reset_index(inplace=True)

In [15]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df['geometry_BG'])

BG_df.reset_index(inplace=True)

In [16]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'yearOfLoss_1990_2021']).index.map(zipcode_df['geometry_zipcode'])

zipcode_df.reset_index(inplace=True)

In [17]:
# Initial mapping with multi-index
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

state_df.reset_index(inplace=True)

In [18]:
# Initial mapping with multi-index
County_df.set_index(['CountyID'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode']).index.map(County_df['geometry_county'])

# Resetting the index of County_df (return to multi-index)
County_df.reset_index(inplace=True)

In [19]:
# Initial mapping with multi-index
Tract_df.set_index(['censusTractID'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract']).index.map(Tract_df['geometry_tract'])

# Resetting the index of County_df (return to multi-index)
Tract_df.reset_index(inplace=True)

### Adding the inconsistent flags to the dataset

In [41]:
df_1_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_1.parquet.gzip')
df_2_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_2.parquet.gzip')
df_3_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_3.parquet.gzip')
df_4_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_4.parquet.gzip')
df_5_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_5.parquet.gzip')
df_6_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_6.parquet.gzip')

In [42]:
df_1_inconsistent = df_1_inconsistent.drop_duplicates()
df_2_inconsistent = df_2_inconsistent.drop_duplicates()
df_3_inconsistent = df_3_inconsistent.drop_duplicates()
df_4_inconsistent = df_4_inconsistent.drop_duplicates()
df_5_inconsistent = df_5_inconsistent.drop_duplicates()
df_6_inconsistent = df_6_inconsistent.drop_duplicates()

In [43]:
# Initial mapping with multi-index
df_1_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

df_geographic_unique['zipInconsistent'] = df_geographic_unique.drop_duplicates(
                    subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 
                    'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss_1990_2021']).set_index(
                    ['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 
                     'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['zipInconsistent'])


df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['zipInconsistent'])
df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['cbgInconsistent'])
df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['tractInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['stateInconsistent'])
df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['countyInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['latlongInconsistent'])

# Resetting the index of County_df (return to multi-index)
df_1_inconsistent.reset_index(inplace=True)

In [44]:
df_2_inconsistent.head()

,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss_2000_2021,state,cbgInconsistent,tractInconsistent,countyInconsistent,stateInconsistent,latlongInconsistent,multiple,noOverlap,oneWrong,year,lat_long_geometry,state_geometry,county_geometry
0,17104.0,42043,42043023400,420430234001,40.3,-76.9,None,PA,0,0,0,0,1,0,0,1,1990.0,Wrong,None,None
1,70345.0,22057,22057021100,220570211001,29.6,-90.3,None,LA,1,0,0,0,1,1,0,0,1990.0,None,None,None
2,8008.0,34029,34029739000,340297390002,39.7,-74.1,None,NJ,0,0,0,0,1,0,0,1,1990.0,Wrong,None,None
3,8008.0,34029,34029739000,340297390005,39.6,-74.2,None,NJ,1,0,0,0,1,1,0,0,1990.0,None,None,None
5,29417.0,45019,45019002101,450190021014,32.6,-80.2,None,SC,0,0,0,0,1,0,0,1,1990.0,Wrong,None,None


In [45]:
# Initial mapping with multi-index
df_2_inconsistent.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude'], inplace=True)

df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_2_inconsistent['cbgInconsistent'])
df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_2_inconsistent['tractInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])

# Resetting the index of County_df (return to multi-index)
df_2_inconsistent.reset_index(inplace=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
# Initial mapping with multi-index
df_3_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['zipInconsistent'])
df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['cbgInconsistent'])
df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['tractInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['stateInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['latlongInconsistent'])

# Resetting the index of County_df (return to multi-index)
df_3_inconsistent.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
df_4_inconsistent.set_index(['censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude'], inplace=True)

df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_4_inconsistent['cbgInconsistent'])
df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_4_inconsistent['tractInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_4_inconsistent['stateInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
                            'latitude', 'longitude']).index.map(df_4_inconsistent['latlongInconsistent'])

# Resetting the index of County_df (return to multi-index)
df_4_inconsistent.reset_index(inplace=True)

In [34]:
# Initial mapping with multi-index
df_5_inconsistent.set_index(['reportedZipCode', 'state', 'countyCode',
                            'latitude', 'longitude', 'year'], inplace=True)

df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['zipInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['stateInconsistent'])
df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['countyInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['latlongInconsistent'])

# Resetting the index of County_df (return to multi-index)
df_5_inconsistent.reset_index(inplace=True)

In [35]:
# # Initial mapping with multi-index
# df_6_inconsistent.set_index(['state', 'countyCode',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])

# # Resetting the index of County_df (return to multi-index)
# df_6_inconsistent.reset_index(inplace=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [36]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss_1990_2021,geometry_lat_long,geometry_BG,geometry_zipcode,geometry_state,geometry_county,geometry_tract,zipInconsistent,cbgInconsistent,tractInconsistent,stateInconsistent,countyInconsistent,latlongInconsistent
0,CA,92056,06073,06073018512,060730185123,33.2,-117.3,1990,"POLYGON ((-117.35000 33.15000, -117.25000 33.1...","POLYGON ((-117.33340 33.20936, -117.33334 33.2...",None,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...","POLYGON ((-117.43743 33.17953, -117.44954 33.1...","POLYGON ((-117.33340 33.20936, -117.33334 33.2...",NaN,NaN,NaN,NaN,NaN,NaN
1,LA,70131,22071,22071000616,220710006163,29.9,-90.0,2000,"POLYGON ((-90.05000 29.85000, -89.95000 29.850...","POLYGON ((-90.00180 29.91346, -90.00172 29.913...","POLYGON ((-90.01051 29.94005, -90.00980 29.939...","MULTIPOLYGON (((-88.86770 29.86155, -88.86566 ...","POLYGON ((-90.14007 29.94790, -90.13765 29.951...","POLYGON ((-90.00180 29.91346, -90.00172 29.913...",NaN,NaN,NaN,NaN,NaN,NaN
2,FL,32566,12113,12113010815,121130108152,30.4,-86.9,1990,"POLYGON ((-86.95000 30.35000, -86.85000 30.350...","POLYGON ((-87.03397 30.38297, -87.03395 30.382...",None,"MULTIPOLYGON (((-80.62717 25.00040, -80.62600 ...","POLYGON ((-87.29836 30.82735, -87.29860 30.827...","POLYGON ((-87.04790 30.39801, -87.04592 30.407...",NaN,NaN,NaN,NaN,NaN,NaN
3,SC,29902,45013,45013000700,450130007002,32.4,-80.7,1990,"POLYGON ((-80.75000 32.35000, -80.65000 32.350...","POLYGON ((-80.69041 32.42965, -80.69039 32.429...",None,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...","POLYGON ((-80.81918 32.48594, -80.81943 32.487...","POLYGON ((-80.70786 32.41190, -80.70783 32.412...",NaN,NaN,NaN,NaN,NaN,NaN
4,FL,32940,12009,12009063107,120090631073,28.3,-80.7,1990,"POLYGON ((-80.75000 28.25000, -80.65000 28.250...","POLYGON ((-80.71503 28.24477, -80.71440 28.244...",None,"MULTIPOLYGON (((-80.62717 25.00040, -80.62600 ...","POLYGON ((-80.86158 28.24656, -80.86155 28.247...","POLYGON ((-80.71826 28.26372, -80.71821 28.264...",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#TODO: for all NA values change it to 1
#TODO: create 'multipleInconsistent' column where add all flags
#TODO: for state = 'UN', make it N
#TODO (Not here but later): for missing geometries, change flag to M

### Creating new geometry

In [ ]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract','censusBlockGroupFips', 'latitude', 
                                        'longitude', 'state', 'new_geographic_unit',  'year', 'geometry_zipcode',
                                       'geometry_county', 'geometry_tract','geometry_BG','geometry_lat_long','geometry_state', 
                                        'geometry_new_unit', 'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 
                                        'stateInconsistent', 'latlongInconsistent', 'zipcodeInconsistent','multipleInconsistent'])

empty = Polygon()

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_1990_2021']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    cbgInconsistent = row_unit['cbgInconsistent']
    tractInconsistent = row_unit['tractInconsistent']
    countyInconsistent = row_unit['countyInconsistent']
    stateInconsistent = row_unit['stateInconsistent']
    latlongInconsistent = row_unit['latlongInconsistent']
    zipcodeInconsistent = row_unit['zipcodeInconsistent']
    multipleInconsistent = row_unit['multipleInconsistent']
    
    if ((cbgInconsistent == 0) & (latlongInconsistent == 0) & (zipcodeInconsistent == 0)):
        intersection_geometry = lat_long_geometry.intersection(bg_geometry).intersection(zipcode_geometry)
    
    elif ((cbgInconsistent == 0) & (latlongInconsistent == 0) & (zipcodeInconsistent == 1)):
        intersection_geometry = lat_long_geometry.intersection(bg_geometry)
        
    elif ((cbgInconsistent == 1) & (latlongInconsistent == 0) & (zipcodeInconsistent == 0)):
        intersection_geometry = lat_long_geometry.intersection(zipcode_geometry)
    
    elif ((cbgInconsistent == 1) & (latlongInconsistent == 0) & (zipcodeInconsistent == 1)):
        intersection_geometry = lat_long_geometry
        
    else:
        intersection_geometry = empty
    
    new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):03}{'0' if long >= 0 else '1'}{int(abs(long) * 10):03}{bg_id:012}{zipcode:05}{year:04}"
    
    new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
        'reportedZipCode': [zipcode],
        'countyCode': [county_id],
        'censusTract': [tract_id],
        'censusBlockGroupFips': [bg_id],
        'latitude': [lat],
        'longitude': [long],
        'state': [state],
        'new_geographic_unit': [new_geographic_unit]
        'year': [year],
        'geometry_zipcode': [zipcode_geometry],
        'geometry_county': [county_geometry],
        'geometry_tract': [tract_geometry],
        'geometry_BG': [bg_geometry],
        'geometry_lat_long': [lat_long_geometry],
        'geometry_state': [state_geometry],
        'geometry_new_unit': [intersection_geometry],
        'cbgInconsistent': [cbgInconsistent],
        'tractInconsistent': [tractInconsistent],
        'countyInconsistent': [countyInconsistent],
        'stateInconsistent': [stateInconsistent],
        'latlongInconsistent': [latlongInconsistent],
        'zipcodeInconsistent': [zipcodeInconsistent],
        'multipleInconsistent': [multipleInconsistent]
        })], ignore_index=True)

## Visualization

In [ ]:
visualize_NC = lat_long_df[(lat_long_df['state'] == 'NC') & (lat_long_df['longitude'] < 174.0) & (lat_long_df['longitude'] < 174.0) & (lat_long_df['latitude'] < 40.0)]

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using Latitude-Longitude geometries')

plt.show()

In [ ]:
visualize_NC = BG_df[(BG_df['state'] == 'NC') & (BG_df['year'] == 2010)]
# Find the centroid for each geometry
centroids = visualize_NC.geometry.centroid

# Filter the rows where the y-coordinate (latitude) of the centroid is greater than 42
visualize_NC = visualize_NC[centroids.y < 37]
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using Census BlockGroup geometries for 2010')

plt.show()

In [ ]:
visualize_NC = zipcode_df[(zipcode_df['state'] == 'NC') & (zipcode_df['year'] == 2010)]

centroids = visualize_NC.geometry.centroid

# Filter the rows where the y-coordinate (latitude) of the centroid is greater than 42
visualize_NC = visualize_NC[centroids.y < 37]
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using Zipcode geometries for 2010')

plt.show()

In [ ]:
visualize_NC = new_unit_df[(zipcode_df['state'] == 'NC') & (new_unit_df['year'] == 2010)]

centroids = visualize_NC.geometry.centroid

# Filter the rows where the y-coordinate (latitude) of the centroid is greater than 42
visualize_NC = visualize_NC[centroids.y < 37]
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using new_geographic_unit geometries for 2010')

plt.show()